In [1]:
from typing import List
from sympy import root
from tree_sitter import Language, Parser, Node

import random

import os
from datasets import load_dataset, Dataset, load_from_disk
from statistics import python_dict, java_dict, go_dict
from tqdm.notebook import tqdm
from colorama import Fore, Back, Style
import pandas as pd
import javalang

from process_data.RobustCodeSumGo import GoParser
import parso

from utils import ParserUtils,DataUtils
import os

os.chdir("/data2/huchao/11.novels/bigcode-evaluation-harness")

In [2]:


DATASET_PATH = "/data2/huchao/11.novels/download/code_x_glue_ct_code_to_text"

dataset = load_dataset(DATASET_PATH, "go")["test"]
print(f"load data from code search net, size: {dataset.shape}")
self_lang_str = "go"
self_langurage = Language("/data2/huchao/11.novels/bigcode-evaluation-harness/process_data/build/my-languages.so", self_lang_str)
self_parser = Parser()
self_parser.set_language(self_langurage)

/home/huchao/micromamba/lib/python3.9/site-packages/datasets/load.py:922: FutureWarning: The repository for code_x_glue_ct_code_to_text contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /data2/huchao/11.novels/download/code_x_glue_ct_code_to_text/code_x_glue_ct_code_to_text.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


load data from code search net, size: (8122, 12)


In [ ]:
# init the parser

## FNE: 替换函数名为v0


In [6]:
row = dataset[10]
code_string = row["code"]
doc_string = row['docstring']
print(doc_string)
doc_tokens = row['docstring_tokens']
print(' '.join(doc_tokens))
# print(code_string)

tree = self_parser.parse(bytes(code_string, "utf8"))
root_node = tree.root_node
from process_data.RobustCodeSumGo import GoParser

identifier_node = GoParser.get_method_identifier_node(root_node)

new_code_string = ParserUtils.replace_node_str(identifier_node, code_string, "v0")

DataUtils.display_before_and_after(code_string, new_code_string)

// pick picks a chan for sending the given message. The picked chan and the picked chan
// string name are returned.
pick picks a chan for sending the given message . The picked chan and the picked chan string name are returned .
before handle:
func (p *peer) pick(m raftpb.Message) (writec chan<- raftpb.Message, picked string) {
	var ok bool
	// Considering MsgSnap may have a big size, e.g., 1G, and will block
	// stream for a long time, only use one of the N pipelines to send MsgSnap.
	if isMsgSnap(m) {
		return p.pipeline.msgc, pipelineMsg
	} else if writec, ok = p.msgAppV2Writer.writec(); ok && isMsgApp(m) {
		return writec, streamAppV2
	} else if writec, ok = p.writer.writec(); ok {
		return writec, streamMsg
	}
	return p.pipeline.msgc, pipelineMsg
}
after handle:
func (p *peer) v0(m raftpb.Message) (writec chan<- raftpb.Message, picked string) {
	var ok bool
	// Considering MsgSnap may have a big size, e.g., 1G, and will block
	// stream for a long time, only use one of the N pipe

NameError: name 'DataUtils' is not defined

## OOS：计算交换

In [6]:
from process_data.RobustCodeSum import PythonParsoParser

row = dataset[10]
code_string = row["code"]
print(code_string)

parsed_ast = parso.parse(code_string)
if len(list(parsed_ast.iter_funcdefs())) == 0:
    # continue
    exit(0)
func_node = list(parsed_ast.iter_funcdefs())[0]
identifier_name_to_nodes = PythonParsoParser.get_all_identifers(func_node)




new_code_string = func_node.get_code()
# print(new_code_string)

def get_video_url_from_video_id(video_id):
    """Splicing URLs according to video ID to get video details"""
    # from js
    data = [""] * 256
    for index, _ in enumerate(data):
        t = index
        for i in range(8):
            t = -306674912 ^ unsigned_right_shitf(t, 1) if 1 & t else unsigned_right_shitf(t, 1)
        data[index] = t

    def tmp():
        rand_num = random.random()
        path = "/video/urls/v/1/toutiao/mp4/{video_id}?r={random_num}".format(video_id=video_id,
                                                                              random_num=str(rand_num)[2:])
        e = o = r = -1
        i, a = 0, len(path)
        while i < a:
            e = ord(path[i])
            i += 1
            if e < 128:
                r = unsigned_right_shitf(r, 8) ^ data[255 & (r ^ e)]
            else:
                if e < 2048:
                    r = unsigned_right_shitf(r, 8) ^ data[255 & (r ^ (192 | e >> 6 & 31))]
                    r = unsigned_right_shitf

## IRS: 随机调换标识符名字（使用函数片段中已有的标识符名字）

In [8]:
# 找到所有的identifier_list
# 打乱ide_list的顺序，然后形成对应，确定替换关系(ide -> ide')
# 在code中替换对象，作为混淆结果

from process_data.RobustCodeSum import PythonParsoParser
from process_data.utils import DataUtils

row = dataset[0]
code_string = row["code"]

tree = self_parser.parse(bytes(code_string, "utf8"))
root_node = tree.root_node

# 使用traverse方法来获取所有的identifier
all_ide_list = []
ParserUtils.traverse_type(root_node, all_ide_list, "identifier")

for _ide in all_ide_list:
    print(_ide.text.decode("utf8"), end=", ")
print("*" * 20)
# 继承原来的方法来获取所有对象identifier（主要是要筛去api调用之类的）
target_ide_name_list = GoParser.get_all_target_ide_name(root_node)
print(target_ide_name_list)
print("*" * 20)

# 两者结合得到所有的name和node对应关系
identifier_dict = {}
for _ide in all_ide_list:
    _name = _ide.text.decode("utf8")
    if _name in target_ide_name_list:
        if not _name in identifier_dict:
            identifier_dict[_name] = [_ide]
        else:
            identifier_dict[_name].append(_ide)

# 生成替换字典
# 匹配,使用zip，使用循环替换的方法来做到打乱效果
replace_dict = dict(zip(target_ide_name_list[:-1], target_ide_name_list[1:]))
replace_dict.update({target_ide_name_list[-1]: target_ide_name_list[0]})


# 替换identifier
nodes_info = []
for identifier, replace_str in replace_dict.items():
    nodes = identifier_dict[identifier]
    for node in nodes:
        nodes_info.append((node, replace_str))
new_code_string = ParserUtils.replace_nodes_str(nodes_info, code_string)


DataUtils.display_before_and_after(code_string, new_code_string)

NewSTM, c, apply, so, opts, ctx, c, _, f, so, f, opts, len, opts, f, apply, apply, s, s, opts, f, s, runSTM, mkSTM, c, opts, apply, ********************
['NewSTM', 'c', 'apply', 'so', 'opts', 'f', 's']
********************
before handle:
func NewSTM(c *v3.Client, apply func(STM) error, so ...stmOption) (*v3.TxnResponse, error) {
	opts := &stmOptions{ctx: c.Ctx()}
	for _, f := range so {
		f(opts)
	}
	if len(opts.prefetch) != 0 {
		f := apply
		apply = func(s STM) error {
			s.Get(opts.prefetch...)
			return f(s)
		}
	}
	return runSTM(mkSTM(c, opts), apply)
}
after handle:
func c(apply *v3.Client, so func(STM) error, opts ...stmOption) (*v3.TxnResponse, error) {
	f := &stmOptions{ctx: apply.Ctx()}
	for _, s := range opts {
		s(f)
	}
	if len(f.prefetch) != 0 {
		s := so
		so = func(NewSTM STM) error {
			NewSTM.Get(f.prefetch...)
			return s(NewSTM)
		}
	}
	return runSTM(mkSTM(apply, f), so)
}


## IHR：使用代码高频词替换标识符

In [6]:
# 找到所有的identifier_list
# 对于每一个目标ide,去高频词库中随机找一个词出来替换(ide,random_word)
# 同时不同ide的替换词不能相同
# 在code中替换对象，作为混淆结果


# 找到所有的identifier_list
# 打乱ide_list的顺序，然后形成对应，确定替换关系(ide -> ide')
# 在code中替换对象，作为混淆结果

from process_data.RobustCodeSum import PythonParsoParser
from process_data.utils import DataUtils


row = dataset[784]
code_string = row["code"]

tree = self_parser.parse(bytes(code_string, "utf8"))
root_node = tree.root_node

# 使用traverse方法来获取所有的identifier
all_ide_list = []
ParserUtils.traverse_type(root_node, all_ide_list, "identifier")

# 继承原来的方法来获取所有对象identifier（主要是要筛去api调用之类的）
target_ide_name_list = GoParser.get_all_target_ide_name(root_node)

# 两者结合得到所有的name和node对应关系
identifier_dict = {}
for _ide in all_ide_list:
    _name = _ide.text.decode("utf8")
    if _name in target_ide_name_list:
        if not _name in identifier_dict:
            identifier_dict[_name] = [_ide]
        else:
            identifier_dict[_name].append(_ide)


# 获取高频词数组
high_freq_list = [word[0] for word in go_dict]


# 对所有identifier随机匹配一个高频词，且不能有重复的
# 做法是，先shuffle high_freq_dict，然后按顺序匹配

# shuffle
random.shuffle(high_freq_list)

# 使用匹配的方式，生成替换字典
replace_dict = {}
for identifier in target_ide_name_list:
    replace_dict[identifier] = high_freq_list.pop()


# 替换identifier
nodes_info = []
for identifier, replace_str in replace_dict.items():
    nodes = identifier_dict[identifier]
    for node in nodes:
        nodes_info.append((node, replace_str))
new_code_string = ParserUtils.replace_nodes_str(nodes_info, code_string)


DataUtils.display_before_and_after(code_string, new_code_string)

before handle:
func (s *levelHandler) deleteTables(toDel []*table.Table) error {
	s.Lock() // s.Unlock() below

	toDelMap := make(map[uint64]struct{})
	for _, t := range toDel {
		toDelMap[t.ID()] = struct{}{}
	}

	// Make a copy as iterators might be keeping a slice of tables.
	var newTables []*table.Table
	for _, t := range s.tables {
		_, found := toDelMap[t.ID()]
		if !found {
			newTables = append(newTables, t)
			continue
		}
		s.totalSize -= t.Size()
	}
	s.tables = newTables

	s.Unlock() // Unlock s _before_ we DecrRef our tables, which can be slow.

	return decrRefs(toDel)
}
after handle:
func (q *levelHandler) deleteTables(tx []*table.Table) error {
	q.Lock() // s.Unlock() below

	b := make(map[uint64]struct{})
	for res, t := range tx {
		b[t.ID()] = struct{}{}
	}

	// Make a copy as iterators might be keeping a slice of tables.
	var newTables []*table.Table
	for res, t := range q.tables {
		res, err := b[t.ID()]
		if !err {
			newTables = append(newTables, t)
			continue
		}


## IOE:标识符变成v0，v1…，函数名也会变

In [26]:
# 找到所有的identifier_list
# 将其中的100%作为替换目标
# 替换identifier_list，每个ide给予一个对象(v_n)
# 在code中替换对象，作为混淆结果

from process_data.RobustCodeSum import PythonParsoParser
from process_data.utils import DataUtils

row = dataset[323]
code_string = row["code"]
# print(code_string)

tree = self_parser.parse(bytes(code_string, "utf8"))
root_node = tree.root_node

# 使用traverse方法来获取所有的identifier
all_ide_list = []
ParserUtils.traverse_type(root_node, all_ide_list, "identifier")
# for _ide in all_ide_list:
#     print(_ide.text.decode("utf8"))
# 继承原来的方法来获取所有对象identifier（主要是要筛去api调用之类的）
target_ide_name_list = []
nodes = [root_node]
while len(nodes) > 0:
    cur_node = nodes.pop(0)
    if cur_node.type == "ERROR":
        print(code_string)
        continue

    if (
        cur_node.type == "variable_declarator"
        or cur_node.type == "formal_parameter"
        or cur_node.type == "method_declaration"
        or cur_node.type == "constructor_declaration"
        or cur_node.type == "assignment_expression"
    ):
        for child in cur_node.children:
            if child.type == "identifier":
                identifier_string = child.text.decode("utf8")
                if not identifier_string in target_ide_name_list:
                    target_ide_name_list.append(identifier_string)
    for child in cur_node.children:
        nodes.append(child)

# 两者结合得到所有的name和node对应关系
identifier_dict = {}
for _ide in all_ide_list:
    _name = _ide.text.decode("utf8")
    if _name in target_ide_name_list:
        if not _name in identifier_dict:
            identifier_dict[_name] = [_ide]
        else:
            identifier_dict[_name].append(_ide)
# for key in identifier_dict:
#     print(key, len(identifier_dict[key]))

# 生成替换字典
replace_dict = {}
for i, ide_name in enumerate(target_ide_name_list):
    replace_dict[ide_name] = f"v{i}"

# 替换identifier
nodes_info = []
for identifier, replace_str in replace_dict.items():
    nodes = identifier_dict[identifier]
    for node in nodes:
        nodes_info.append((node, replace_str))
new_code_string = ParserUtils.replace_nodes_str(nodes_info, code_string)
DataUtils.display_before_and_after(code_string, new_code_string)

before handle:
private static boolean overrides(Method a, Method b) {
    // See JLS section 8.4.8.1
    int modifiers = b.getModifiers();
    if (Modifier.isPublic(modifiers) || Modifier.isProtected(modifiers)) {
      return true;
    }
    if (Modifier.isPrivate(modifiers)) {
      return false;
    }
    // b must be package-private
    return a.getDeclaringClass().getPackage().equals(b.getDeclaringClass().getPackage());
  }
after handle:
ic boolean v0(Method v1, Method v2) {
    // See JLS section 8.4.8.1
    int v3 = v2.getModifiers();
    if (Modifier.isPublic(v3) || Modifier.isProtected(v3)) {
      return true;
    }
    if (Modifier.isPrivate(v3)) {
      return false;
    }
    // b must be package-private
    return v1.getDeclaringClass().getPackage().equals(v2.getDeclaringClass().getPackage());
 


## HVI:加入无意义的高频次的变量声明


In [21]:
from process_data.utils import DataUtils

row = dataset[54]
code_string = row["code"]

print(code_string)

tree = self_parser.parse(bytes(code_string, "utf8"))
root_node = tree.root_node


# 继承原来的方法来获取所有对象identifier（主要是要筛去api调用之类的）
target_ide_name_list = GoParser.get_all_target_ide_name(root_node)


insert_list = []
insert_num = 3
for i in range(len(java_dict)):
    if not java_dict[i][0] in target_ide_name_list:
        insert_list.append(java_dict[i][0])
        if len(insert_list) == insert_num:
            break

func_body_node = GoParser.get_method_body_node(root_node)
candidates_list = []
for node in func_body_node.children:
    if len(node.type) > 2:
        candidates_list.append(node)
insert_node = random.choice(candidates_list)
print(f"insert node = {insert_node.text}")

# 插入在这个child的前面
# 获取这个位置的缩进
insert_indent_count = ParserUtils.get_node_indent_count(insert_node, code_string)
print(f"insert indent = {insert_indent_count}")


# 制作插入体
insert_str = ""
for var in insert_list:
    insert_str += f"{' '*insert_indent_count}var {var} int = {random.randint(0,100)}\n"
# 最后的位置去掉换行
insert_str = insert_str[:-1]
print(insert_str)


# 获取这个位置的行号
insert_position = insert_node.start_point[0]
print(f"insert line number = {insert_position}")

# 进行插入
body_code_lines = code_string.split("\n")
body_code_lines.insert(insert_position, insert_str)
new_code_string = "\n".join(body_code_lines)

DataUtils.display_before_and_after(code_string, new_code_string)

func (r *raft) bcastAppend() {
	r.forEachProgress(func(id uint64, _ *Progress) {
		if id == r.id {
			return
		}

		r.sendAppend(id)
	})
}
insert node = b'r.forEachProgress(func(id uint64, _ *Progress) {\n\t\tif id == r.id {\n\t\t\treturn\n\t\t}\n\n\t\tr.sendAppend(id)\n\t})'
insert indent = 4
    var i int = 96
    var value int = 12
    var result int = 79
insert line number = 1
before handle:
func (r *raft) bcastAppend() {
	r.forEachProgress(func(id uint64, _ *Progress) {
		if id == r.id {
			return
		}

		r.sendAppend(id)
	})
}
after handle:
func (r *raft) bcastAppend() {
    var i int = 96
    var value int = 12
    var result int = 79
	r.forEachProgress(func(id uint64, _ *Progress) {
		if id == r.id {
			return
		}

		r.sendAppend(id)
	})
}


### 检测这样获取method body的方法是否总是成功

结果是总是成功

In [22]:

for row in dataset:
    code_string = row["code"]
    code_string = JavaParser.wrap_java_code(code_string)

    tree = self_parser.parse(bytes(code_string, "utf8"))
    root_node = tree.root_node
    func_body_node = (
        root_node.children[0].children[-1].children[1].child_by_field_name("body")
    )
    if func_body_node == None:
        print("error~!")
        break

## DBI:插入死分支


In [7]:
test_idx = 14

In [38]:
from numpy import insert
from process_data.utils import DataUtils

print(f"idx = {test_idx}")
row = dataset[test_idx]
test_idx += 100

code_string = row["code"]
print(code_string)

tree = self_parser.parse(bytes(code_string, "utf8"))
root_node = tree.root_node
func_body_node = root_node.children[0].child_by_field_name("body")
print(func_body_node.text.decode("utf8"))

# print(func_body_node.text.decode("utf8"))

insert_indent_count = 4
print(f"insert indent = {insert_indent_count}")

insert_body = DataUtils.get_dead_if_body("go", " " * insert_indent_count)
print(insert_body.encode())
# print(insert_body)


# 给原body做缩进
code_lines = code_string.split("\n")
## 确定起始位置
move_start_line = func_body_node.start_point[0] + 1
move_end_line = func_body_node.end_point[0] + 1
## 加tab移动
for i in range(move_start_line, move_end_line):
    code_lines[i] = "    " + code_lines[i]

random_case = random.randint(0, 1)
random_case = 0
if random_case == 0:
    # 插入 if true & else
    if_stmt = f"{' '*insert_indent_count}if true {{"
    code_lines.insert(move_start_line, if_stmt)
    else_stmt = f'{" " *insert_indent_count}else {{{insert_body}\n{" "*insert_indent_count}}}\n}}'
    print(len(code_lines), move_end_line)
    code_lines.insert(move_end_line + 1, else_stmt)
else:
    ## 插入 if false & else
    front_stmt = f"{' '*insert_indent_count}if false {{{insert_body}\n{' '*insert_indent_count}}} else {{"
    code_lines.insert(move_start_line, front_stmt)
    code_lines.insert(move_end_line + 2, "}")
# print("\n".join(code_lines))
new_code_string = "\n".join(code_lines)
DataUtils.display_before_and_after(code_string, new_code_string)

idx = 3014
func NotNil(tb testing.TB, object interface{}, msgAndArgs ...interface{}) {
	tb.Helper()
	success := true

	if object == nil {
		success = false
	} else {
		value := reflect.ValueOf(object)
		kind := value.Kind()
		if kind >= reflect.Chan && kind <= reflect.Slice && value.IsNil() {
			success = false
		}
	}

	if !success {
		fatal(tb, msgAndArgs, "Expected value not to be nil.")
	}
}
{
	tb.Helper()
	success := true

	if object == nil {
		success = false
	} else {
		value := reflect.ValueOf(object)
		kind := value.Kind()
		if kind >= reflect.Chan && kind <= reflect.Slice && value.IsNil() {
			success = false
		}
	}

	if !success {
		fatal(tb, msgAndArgs, "Expected value not to be nil.")
	}
}
insert indent = 4
b'\n        return storeTrustedKey(ks.LocalRootPath, r)'
18 18
19 18
before handle:
func NotNil(tb testing.TB, object interface{}, msgAndArgs ...interface{}) {
	tb.Helper()
	success := true

	if object == nil {
		success = false
	} else {
		value := reflect.ValueOf(objec

### 检查body最后的}是不是独立成行

结论：所有数据的最后一个}都是独立成行的


In [20]:
### 检查body最后的}是不是独立成行

for idx, row in enumerate(dataset):
    code_string = row["code"]
    tree = self_parser.parse(bytes(code_string, "utf8"))
    root_node = tree.root_node
    func_body_node = root_node.children[0].child_by_field_name("body")
    body_code_string = func_body_node.text.decode("utf8")

    body_code_lines = body_code_string.split("\n")
    last_line = body_code_lines[-1]
    last_line_strip = last_line.replace("}", " ").strip()
    if last_line_strip != "":
        print(f"idx = {idx}")
        print(last_line.encode())
        # print(sttp.encode())
        # print(func_body_node.text.decode("utf8"))
        # break

### 检查body开头的{的问题

结论：所有数据的开头的{都是独立成行的

In [21]:
for idx, row in enumerate(dataset):
    code_string = row["code"]
    tree = self_parser.parse(bytes(code_string, "utf8"))
    root_node = tree.root_node
    func_body_node = root_node.children[0].child_by_field_name("body")
    body_code_string = func_body_node.text.decode("utf8")
    body_code_lines = body_code_string.split("\n")
    first_line = body_code_lines[0]
    first_strip = first_line.strip()
    if first_strip != "{":
        print(f"idx = {idx}")
        print(first_strip.encode())

# parso Test

In [22]:
import parso

from process_data.RobustCodeSum import PythonParsoParser

row = dataset[0]
# print(row["code"])
code = row["code"]
parsed_ast = parso.parse(code, error_recovery=False, version="3.7")
code_tokens, comment_tokens = [], []

func_nodes = list(parsed_ast.iter_funcdefs())

func_node = func_nodes[0]
doc_node = func_node.get_doc_node()

body_block = PythonParsoParser.find_function_body_block(func_node)
body_block_string = PythonParsoParser.get_body_block_string(body_block)
# 每一行加一个\t
body_block_string = "\t" + body_block_string.replace("\n", "\n\t")

    rawurl = []
    dom = parseString(xml_data)
    for node in dom.getElementsByTagName('durl'):
        url = node.getElementsByTagName('url')[0]
        rawurl.append(url.childNodes[0].data)
    return rawurl
	    rawurl = []
	    dom = parseString(xml_data)
	    for node in dom.getElementsByTagName('durl'):
	        url = node.getElementsByTagName('url')[0]
	        rawurl.append(url.childNodes[0].data)
	    return rawurl


# 测试从输出位置load data文件


In [3]:
# load data
from datasets import load_dataset,load_from_disk

reload_data = load_from_disk('../local_data/second/semantic/IRS/python/CSN')
print(reload_data.shape)

(14917, 12)


In [3]:
async def renew_lock(self):
    """Renew the message lock.

    This will maintain the lock on the message to ensure
    it is not returned to the queue to be reprocessed. In order to complete (or otherwise settle)
    the message, the lock must be maintained. Messages received via ReceiveAndDelete mode are not
    locked, and therefore cannot be renewed. This operation can also be performed as an asynchronous
    background task by registering the message with an `azure.servicebus.aio.AutoLockRenew` instance.
    This operation is only available for non-sessionful messages.

    :raises: TypeError if the message is sessionful.
    :raises: ~azure.servicebus.common.errors.MessageLockExpired is message lock has already expired.
    :raises: ~azure.servicebus.common.errors.SessionLockExpired if session lock has already expired.
    :raises: ~azure.servicebus.common.errors.MessageAlreadySettled is message has already been settled.
    """
    if hasattr(self._receiver, "locked_until"):
        raise TypeError(
            "Session messages cannot be renewed. Please renew the Session lock instead."
        )
    self._is_live("renew")
    expiry = await self._receiver._renew_locks(
        self.lock_token
    )  # pylint: disable=protected-access
    self._expiry = datetime.datetime.fromtimestamp(expiry[b"expirations"][0] / 1000.0)
    
    

2


# 测试删除Java注释

检查后发现CodeSearchNet中的Java数据的tokens数据中也留有注释

所以决定这里也不做删除注释操作了


In [24]:
from process_data.RobustCodeSum import PythonParsoParser

row = dataset[323]
code_string = row["code"]
code_tokens = row['code_tokens']
print(' '.join(code_tokens))
# print(code_string)

tree = self_parser.parse(bytes(code_string, "utf8"))
root_node = tree.root_node
comments_node = []
ParserUtils.traverse_type(root_node, comments_node, "comment")
for comment in comments_node:
    print(comment.text.decode("utf8"))

private static boolean overrides ( Method a , Method b ) { // See JLS section 8.4.8.1 int modifiers = b . getModifiers ( ) ; if ( Modifier . isPublic ( modifiers ) || Modifier . isProtected ( modifiers ) ) { return true ; } if ( Modifier . isPrivate ( modifiers ) ) { return false ; } // b must be package-private return a . getDeclaringClass ( ) . getPackage ( ) . equals ( b . getDeclaringClass ( ) . getPackage ( ) ) ; }
